In [3]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import random

from scipy import eye, asarray, dot, sum, diag
from scipy.linalg import svd
from mpl_toolkits.basemap import Basemap
import matplotlib as mpl
from matplotlib import cm
from colorspacious import cspace_converter
from collections import OrderedDict
from netCDF4 import Dataset
from numpy import linspace
from numpy import meshgrid


import PCA_functions as pf
import Extreme_functions as ef

from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, CMIknn
import tigramite.data_processing as pp

from Data import Data
from datetime import datetime

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import itertools
from collections import Counter
import pickle

from numpy import linalg as LA

import matplotlib.cm as cm

from scipy.special import inv_boxcox
from scipy import stats

from matplotlib import animation

import scipy.stats as st

import reverse_geocoder as rg

from pandas_datareader import wb
import scipy.stats as st

import Rung as rung

from pandas_datareader import wb

import seaborn as sns


import seaborn as sns
import feature_finder_keiko as ff
#import feature_finder_f as ff1

from scipy import signal

plt.rcParams["figure.figsize"] = (20,10)

In [4]:
train_start = 2000
validation_end = 2010
f = 12
n_components_sst = 8
tau =12

In [5]:
temporal_limits = {"time_min":datetime(train_start, 1, 1, 0, 0),"time_max":datetime(validation_end, 12, 1, 0, 0)}
#original_count, count = ff.drought_timeseries("../ET_gamma_18912015.npy",train_start[ijz],validation_end[ijz])
#data_sst, ts, V, df_sst, avg, std = ff.PCA_computer_rotated('../../../nc/sst.mnmean.nc', "sst",temporal_limits, n_components_sst, -9.96921e+36)

In [6]:
sst = Data('../../../nc/sst.mnmean.nc',"sst",temporal_limits, missing_value= -9.96921e+36)

result = sst.get_data()
lon_sst_list = sst.get_lon_list()
lat_sst_list = sst.get_lat_list()
lon_sst = sst.get_lon()
lat_sst = sst.get_lat()

result_sst, avgs, stds = pf.deseasonalize_avg_std(np.array(result))
result_sst = signal.detrend(result_sst, axis=0)
weights = np.sqrt(np.abs(np.cos(np.array(lat_sst_list)* math.pi/180)))
for i in range(len(weights)):
    result_sst[:,i] = weights[i] * result_sst[:,i]

data_sst = pd.DataFrame(result_sst)

V, U, S, ts, eig, explained, max_comps = rung.pca_svd(data_sst,truncate_by='max_comps', max_comps=n_components_sst)


Vr, Rot = rung.varimax(V)
Vr = rung.svd_flip(Vr)

# Get explained variance of rotated components
s2 = np.diag(S)**2 / (ts.shape[0] - 1.)

# matrix with diagonal containing variances of rotated components
S2r = np.dot(np.dot(np.transpose(Rot), np.matrix(np.diag(s2))), Rot)
expvar = np.diag(S2r)

sorted_expvar = np.sort(expvar)[::-1]
# s_orig = ((Vt.shape[1] - 1) * s2) ** 0.5

# reorder all elements according to explained variance (descending)
nord = np.argsort(expvar)[::-1]
Vr = Vr[:, nord]

# Get time series of UNMASKED data
comps_ts = np.matmul(np.array(data_sst),Vr)

df_sst = pd.DataFrame({"lons":lon_sst_list,"lats":lat_sst_list})

lon_temp = df_sst["lons"].values
lon_temp[lon_temp > 180] = lon_temp[lon_temp > 180] -360
df_sst["lons"].vlues = lon_temp

In [7]:
ts = np.array(ts)

In [8]:
dataframe = pp.DataFrame(ts)
cond_ind_test = ParCorr()
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cond_ind_test)

In [9]:
results = pcmci.run_pcmci(tau_min=1, tau_max=3, pc_alpha=None)

In [10]:
pcmci.print_significant_links(p_matrix=results['p_matrix'],
                                     val_matrix=results['val_matrix'],
                                     alpha_level=0.05)


## Significant links at alpha = 0.05:

    Variable 0 has 3 link(s):
        (0 -1): pval = 0.00000 | val = 0.737
        (4 -1): pval = 0.02949 | val = -0.200
        (7 -2): pval = 0.03777 | val = -0.192

    Variable 1 has 5 link(s):
        (1 -1): pval = 0.00000 | val = 0.758
        (1 -2): pval = 0.00067 | val = -0.307
        (1 -3): pval = 0.00113 | val = 0.296
        (4 -1): pval = 0.02756 | val = 0.201
        (0 -1): pval = 0.04654 | val = 0.185

    Variable 2 has 3 link(s):
        (2 -1): pval = 0.00000 | val = 0.816
        (2 -2): pval = 0.00096 | val = -0.294
        (0 -2): pval = 0.02702 | val = -0.204

    Variable 3 has 1 link(s):
        (3 -1): pval = 0.00000 | val = 0.690

    Variable 4 has 3 link(s):
        (4 -1): pval = 0.00000 | val = 0.733
        (6 -2): pval = 0.00916 | val = 0.236
        (2 -1): pval = 0.03961 | val = 0.187

    Variable 5 has 1 link(s):
        (5 -1): pval = 0.00000 | val = 0.741

    Variable 6 has 2 link(s):
        (6 -1): pva

In [28]:
    pq_matrix = results['p_matrix']
    val_matrix = results['val_matrix']

In [30]:
pq_matrix.shape

(8, 8, 4)

In [32]:
val_matrix.shape

(8, 8, 4)

In [40]:
alpha_level = 0.05

In [41]:
    N = pq_matrix.shape[0]

    link_dict = dict()
    for j in range(N):
        # Get the good links
        good_links = np.argwhere(pq_matrix[:, j, 1:] <= alpha_level)
        # Build a dictionary from these links to their values
        links = {(i, -tau - 1): np.abs(val_matrix[i, j, abs(tau) + 1])
                 for i, tau in good_links}
        # Sort by value
        link_dict[j] = sorted(links, key=links.get, reverse=True)
        
    link = np.array(link_dict[0])
    link = link[link[:,0] != 0,:]

In [42]:
link

array([[ 4, -1],
       [ 7, -2]])

In [18]:
alpha_level = 0.05
pq_matrix = results['p_matrix']
val_matrix = results['val_matrix']

In [19]:
q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], fdr_method='fdr_bh')
N = q_matrix.shape[0]
link_dict = dict()
for j in range(N):
    # Get the good links
    good_links = np.argwhere(q_matrix[:, j, 1:] <= alpha_level)
    # Build a dictionary from these links to their values
    links = {(i, -tau - 1): np.abs(val_matrix[i, j, abs(tau) + 1])
             for i, tau in good_links}
    # Sort by value
    link_dict[j] = sorted(links, key=links.get, reverse=True)

link = np.array(link_dict[0])
link = link[link[:,0] != 0,:]

In [20]:
link

array([], shape=(0, 2), dtype=int64)

In [11]:
q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], fdr_method='fdr_bh')

In [13]:
pcmci.print_significant_links(
        p_matrix = results['p_matrix'], 
        q_matrix = q_matrix,
        val_matrix = results['val_matrix'],
        alpha_level = 0.01)


## Significant links at alpha = 0.01:

    Variable 0 has 1 link(s):
        (0 -1): pval = 0.00000 | qval = 0.00000 | val = 0.737

    Variable 1 has 1 link(s):
        (1 -1): pval = 0.00000 | qval = 0.00000 | val = 0.758

    Variable 2 has 1 link(s):
        (2 -1): pval = 0.00000 | qval = 0.00000 | val = 0.816

    Variable 3 has 1 link(s):
        (3 -1): pval = 0.00000 | qval = 0.00000 | val = 0.690

    Variable 4 has 1 link(s):
        (4 -1): pval = 0.00000 | qval = 0.00000 | val = 0.733

    Variable 5 has 1 link(s):
        (5 -1): pval = 0.00000 | qval = 0.00000 | val = 0.741

    Variable 6 has 1 link(s):
        (6 -1): pval = 0.00000 | qval = 0.00000 | val = 0.715

    Variable 7 has 1 link(s):
        (7 -1): pval = 0.00000 | qval = 0.00000 | val = 0.773


In [27]:
pcmci.print_significant_links(
        p_matrix = results['p_matrix'], 
        q_matrix = q_matrix,
        val_matrix = results['val_matrix'],
        alpha_level = 0.05)


## Significant links at alpha = 0.05:

    Variable 0 has 1 link(s):
        (0 -1): pval = 0.00000 | qval = 0.00000 | val = 0.737

    Variable 1 has 3 link(s):
        (1 -1): pval = 0.00000 | qval = 0.00000 | val = 0.758
        (1 -2): pval = 0.00067 | qval = 0.01431 | val = -0.307
        (1 -3): pval = 0.00113 | qval = 0.01811 | val = 0.296

    Variable 2 has 2 link(s):
        (2 -1): pval = 0.00000 | qval = 0.00000 | val = 0.816
        (2 -2): pval = 0.00096 | qval = 0.01677 | val = -0.294

    Variable 3 has 1 link(s):
        (3 -1): pval = 0.00000 | qval = 0.00000 | val = 0.690

    Variable 4 has 1 link(s):
        (4 -1): pval = 0.00000 | qval = 0.00000 | val = 0.733

    Variable 5 has 1 link(s):
        (5 -1): pval = 0.00000 | qval = 0.00000 | val = 0.741

    Variable 6 has 1 link(s):
        (6 -1): pval = 0.00000 | qval = 0.00000 | val = 0.715

    Variable 7 has 2 link(s):
        (7 -1): pval = 0.00000 | qval = 0.00000 | val = 0.773
        (7 -2): pval = 0.0009

In [50]:
    N = q_matrix.shape[0]

    link_dict = dict()
    for j in range(N):
        # Get the good links
        good_links = np.argwhere(q_matrix[:, j, 1:] <= alpha_level)
        # Build a dictionary from these links to their values
        links = {(i, -tau - 1): np.abs(val_matrix[i, j, abs(tau) + 1])
                 for i, tau in good_links}
        # Sort by value
        link_dict[j] = sorted(links, key=links.get, reverse=True)
        
    link = np.array(link_dict[7])
    #link = link[link[:,0] != 0,:]

In [51]:
link

array([[ 7, -1],
       [ 7, -2]])